In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [21]:
def carrera(url):
    response = requests.get(url, allow_redirects=True)

    # Verifica si la solicitud fue exitosa
    if response.status_code == 200:
        # Analiza el contenido HTML con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extrae la tabla (ajusta según la estructura de la página)
        table = soup.find('table')

        # Si se encuentra la tabla, la procesa
        if table:
            # Extrae los encabezados
            headers = [header.get_text() for header in table.find_all('th')]
            
            # Extrae las filas
            rows = []
            for row in table.find_all('tr'):
                columns = row.find_all('td')
                row_data = [col.get_text().strip() for col in columns]
                if row_data:
                    rows.append(row_data)
            
            return headers, rows  # Devuelve los encabezados y las filas
        else:
            print('No se encontró una tabla en la página.')
            return None, None  # Devuelve None si no hay tabla
    else:
        print(f'Error al acceder a la página: {response.status_code}')
        return None, None  # Devuelve None si hay error 

In [22]:
# URL de la página principal de las carreras
url_principal = 'https://admision.unmsm.edu.pe/Website20251/A.html'
response = requests.get(url_principal)

# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    # Analiza el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para almacenar los enlaces de las carreras
    carrera_links = []

    # Encuentra todos los enlaces (a)
    for a_tag in soup.find_all('a'):
        href = a_tag.get('href')
        # Filtra enlaces que contengan '/A/'
        if href and '/A/' in href:
            carrera_links.append(href)

    # Imprimir la lista de enlaces
    print(carrera_links)
else:
    print(f'Error al acceder a la página: {response.status_code}')

['./A/022/0.html', './A/023/0.html', './A/031/0.html', './A/0310/0.html', './A/033/0.html', './A/034/0.html', './A/035/0.html', './A/036/0.html', './A/037/0.html', './A/038/0.html', './A/039/0.html', './A/0611/0.html', './A/0612/0.html', './A/0613/0.html', './A/062/0.html', './A/071/0.html', './A/072/0.html', './A/073/0.html', './A/091/0.html', './A/0911/0.html', './A/0912/0.html', './A/092/0.html', './A/0921/0.html', './A/0922/0.html', './A/093/0.html', './A/0931/0.html', './A/0932/0.html', './A/094/0.html', './A/095/0.html', './A/096/0.html', './A/101/0.html', './A/102/0.html', './A/103/0.html', './A/111/0.html', './A/1111/0.html', './A/1112/0.html', './A/112/0.html', './A/1121/0.html', './A/113/0.html', './A/1131/0.html', './A/114/0.html', './A/1141/0.html', './A/121/0.html', './A/122/0.html', './A/123/0.html', './A/131/0.html', './A/132/0.html', './A/141/0.html', './A/142/0.html', './A/144/0.html', './A/145/0.html', './A/151/0.html', './A/152/0.html', './A/153/0.html', './A/154/0.h

In [23]:
careers_url = []
for links in carrera_links:
    modified = links.replace('.', 'https://admision.unmsm.edu.pe/Website20251', 1)
    careers_url.append(modified)

print(careers_url)

['https://admision.unmsm.edu.pe/Website20251/A/022/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/023/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/031/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0310/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/033/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/034/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/035/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/036/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/037/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/038/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/039/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0611/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0612/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0613/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/062/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/071/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/072/0

In [24]:
# Lista para acumular todas las filas y los encabezados
all_rows = []
all_headers = None  # Para almacenar los encabezados (suponiendo que son los mismos en todas las tablas)

for url in careers_url:
    headers, rows = carrera(url)
    if headers and rows:
        # Almacena los encabezados solo una vez
        if all_headers is None:
            all_headers = headers
        all_rows.extend(rows)  # Agrega las filas a la lista total

# Crea un DataFrame con todos los datos
if all_rows:
    df = pd.DataFrame(all_rows, columns=all_headers)

    def corregir_codificacion(texto):
        return texto.replace('Ã\x8d', 'Í') \
                    .replace('Ã\x93', 'Ó') \
                    .replace('Ã\x81', 'Á') \
                    .replace('Ã\x9a', 'Ú') \
                    .replace('Ã\x89', 'É') \
                    .replace('Ã\x89', 'É') \
                    .replace('Ã\x8dA', 'ÍA') \
                    .replace('Ã\x81A', 'Á')

    # Aplicar corrección en la columna del DataFrame
    df['ESCUELA PROFESIONAL'] = df['ESCUELA PROFESIONAL'].apply(corregir_codificacion)
    #df['ESCUELA SEGUNDA OPCIÓN'] = df['ESCUELA SEGUNDA OPCIÓN'].apply(corregir_codificacion)
    df['APELLIDOS Y NOMBRES'] = df['APELLIDOS Y NOMBRES'].apply(corregir_codificacion)
    # Exportar a CSV
    df.to_csv('postulantes_por_carrera2025I.csv', index=False)
else:
    print("No se encontraron datos para exportar.")

In [19]:
# areas_examen = [
#     {'area_code': 'A', 'career_1': 'MEDICINA HUMANA'},
#     {'area_code': 'A', 'career_1': 'OBSTETRICIA'},
#     {'area_code': 'A', 'career_1': 'ENFERMERÍA'},
#     {'area_code': 'A', 'career_1': 'TECNOLOGÍA MÉDICA'},
#     {'area_code': 'A', 'career_1': 'LABORATORIO CLÍNICO Y ANATOMÍA PATOLÓGICA'},
#     {'area_code': 'A', 'career_1': 'TERAPIA FÍSICA Y REHABILITACIÓN'},
#     {'area_code': 'A', 'career_1': 'RADIOLOGÍA'},
#     {'area_code': 'A', 'career_1': 'TERAPIA OCUPACIONAL'},
#     {'area_code': 'A', 'career_1': 'NUTRICIÓN'},
#     {'area_code': 'A', 'career_1': 'FARMACIA Y BIOQUÍMICA'},
#     {'area_code': 'A', 'career_1': 'CIENCIAS DE LOS ALIMENTOS'},
#     {'area_code': 'A', 'career_1': 'TOXICOLOGÍA'},
#     {'area_code': 'A', 'career_1': 'ODONTOLOGÍA'},
#     {'area_code': 'A', 'career_1': 'MEDICINA VETERINARIA'},
#     {'area_code': 'A', 'career_1': 'PSICOLOGÍA'},
#     {'area_code': 'A', 'career_1': 'PSICOLOGÍA ORGANIZACIONAL Y DE LA GESTIÓN HUMANA'},
#     {'area_code': 'B', 'career_1': 'QUÍMICA'},
#     {'area_code': 'B', 'career_1': 'CIENCIAS BIOLÓGICAS'},
#     {'area_code': 'B', 'career_1': 'GENÉTICA Y BIOTECNOLOGÍA'},
#     {'area_code': 'B', 'career_1': 'MICROBIOLOGÍA Y PARASITOLOGÍA'},
#     {'area_code': 'B', 'career_1': 'FÍSICA'},
#     {'area_code': 'B', 'career_1': 'MATEMÁTICA'},
#     {'area_code': 'B', 'career_1': 'ESTADÍSTICA'},
#     {'area_code': 'B', 'career_1': 'INVESTIGACIÓN OPERATIVA'},
#     {'area_code': 'B', 'career_1': 'COMPUTACIÓN CIENTÍFICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA QUÍMICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA AGROINDUSTRIAL'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA MECÁNICA DE FLUIDOS'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA GEOLÓGICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA GEOGRÁFICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA DE MINAS'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA METALÚRGICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA CIVIL'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA AMBIENTAL'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA INDUSTRIAL'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA TEXTIL Y CONFECCIONES'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA DE SEGURIDAD Y SALUD EN EL TRABAJO'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA ELECTRÓNICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA ELÉCTRICA'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA DE TELECOMUNICACIONES'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA DE SISTEMAS'},
#     {'area_code': 'C', 'career_1': 'INGENIERÍA DE SOFTWARE'},
#     {'area_code': 'E', 'career_1': 'DERECHO'},
#     {'area_code': 'E', 'career_1': 'CIENCIA POLÍTICA'},
#     {'area_code': 'E', 'career_1': 'LITERATURA'},
#     {'area_code': 'E', 'career_1': 'FILOSOFÍA'},
#     {'area_code': 'E', 'career_1': 'LINGÜÍSTICA'},
#     {'area_code': 'E', 'career_1': 'COMUNICACIÓN SOCIAL'},
#     {'area_code': 'E', 'career_1': 'ARTE'},
#     {'area_code': 'E', 'career_1': 'BIBLIOTECOLOGÍA Y CIENCIAS DE LA INFORMACIÓN'},
#     {'area_code': 'E', 'career_1': 'DANZA'},
#     {'area_code': 'E', 'career_1': 'CONSERVACIÓN Y RESTAURACIÓN'},
#     {'area_code': 'E', 'career_1': 'EDUCACIÓN'},
#     {'area_code': 'E', 'career_1': 'EDUCACIÓN INICIAL'},
#     {'area_code': 'E', 'career_1': 'EDUCACIÓN PRIMARIA'},
#     {'area_code': 'E', 'career_1': 'EDUCACIÓN SECUNDARIA'},
#     {'area_code': 'E', 'career_1': 'EDUCACIÓN FÍSICA'},
#     {'area_code': 'E', 'career_1': 'HISTORIA'},
#     {'area_code': 'E', 'career_1': 'SOCIOLOGÍA'},
#     {'area_code': 'E', 'career_1': 'ANTROPOLOGÍA'},
#     {'area_code': 'E', 'career_1': 'ARQUEOLOGÍA'},
#     {'area_code': 'E', 'career_1': 'TRABAJO SOCIAL'},
#     {'area_code': 'E', 'career_1': 'GEOGRAFÍA'},
#     {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN'},
#     {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN DE TURISMO'},
#     {'area_code': 'D', 'career_1': 'ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES'},
#     {'area_code': 'D', 'career_1': 'CONTABILIDAD'},
#     {'area_code': 'D', 'career_1': 'GESTIÓN TRIBUTARIA'},
#     {'area_code': 'D', 'career_1': 'AUDITORÍA EMPRESARIAL Y DEL SECTOR PÚBLICO'},
#     {'area_code': 'D', 'career_1': 'ECONOMÍA'},
#     {'area_code': 'D', 'career_1': 'ECONOMÍA PÚBLICA'},
#     {'area_code': 'D', 'career_1': 'ECONOMÍA INTERNACIONAL'}

# ]


# df_areas = pd.DataFrame(areas_examen)
# df_areas